In [12]:
import Bio
import numpy as np
import math
import random
random.seed()

In [2]:
genes = {"A":300,"B":300,"C":0,"D":0,"E":0,"F":0}
lam = 0.25
alph = 250
ki = 200
ka = 400
hi = 2
ha = 20

In [30]:
def compDs():
    dgAdt = (alph * (1/1+np.power((genes["B"]/ki),hi))) - lam*genes["A"]
    dgBdt = (alph * (1/1+np.power((genes["A"]/ki),hi))) - lam*genes["B"]
    dgCdt = (alph * (1/1+np.power((ka/genes["A"]),ha)) * (1/1+np.power((genes["D"]/ki),hi))) - lam*genes["C"]
    dgDdt = (alph * (1/1+np.power((ka/genes["A"]),ha)) * (1/1+np.power((genes["C"]/ki),hi))) - lam*genes["D"]
    dgEdt = (alph * (1/1+np.power((ka/genes["B"]),ha)) * (1/1+np.power((genes["F"]/ki),hi))) - lam*genes["E"]
    dgFdt = (alph * (1/1+np.power((ka/genes["B"]),ha)) * (1/1+np.power((genes["E"]/ki),hi))) - lam*genes["F"]
    return [dgAdt,dgBdt,dgCdt,dgDdt,dgEdt,dgFdt]

In [31]:
for i in compDs():
    print(i)

737.5
737.5
79084.21380030422
79084.21380030422
79084.21380030422
79084.21380030422


In [26]:
deltt = 1
sigma = 10
tmax = 100
tmin = 0
y = 0
print(y)

0


In [27]:
for t in range(tmin,tmax):
    y = y + (dgAdt*deltt) + (sigma*np.sqrt(deltt)*random.gauss(0,1))
    print(y)

744.7248812168774
1467.2669399070346
2200.8195645883598
2943.38918882433
3672.6489713927285
4411.981641645602
5144.791067773666
5896.012951266111
6633.569721459938
7373.013216147841
8113.420564231192
8856.259268466369
9591.764102822206
10327.623878822817
11049.444223508013
11787.131063753115
12519.876722938976
13266.00586945721
14001.356470155584
14736.062784299418
15485.538170960137
16233.131807703145
16971.57117305306
17700.835898997222
18436.412160343716
19166.373112733912
19890.20499982585
20613.911884567307
21367.917706755175
22095.476865623925
22846.913955755033
23568.513186907163
24300.680487134905
25054.45151341155
25801.86564049667
26535.088094641676
27279.18322985013
28013.179279927204
28738.52380858688
29477.658891180105
30231.797963868146
30977.601649344888
31717.202943570468
32463.178506040564
33208.838722203334
33927.11173357589
34672.7541497274
35421.58619503552
36181.09291305378
36954.87562549602
37681.870617478
38420.05182033414
39152.55327093008
39894.34349674209
4063